# Settings

In [2]:
IN_FILENAME = "../data/processed/formatted.xlsx"
OUT_FILENAME = "../data/processed/calculated.xlsx"

SEPARATOR = "\n"

### adjustments

In [3]:
import re

fmt = re.search("[.][a-z]+$", OUT_FILENAME)
if fmt == None: raise Exception("have to choose a format (xlsx | csv)")
fmt = fmt.group()[1:]

# Prevs

### Imports

In [4]:
import pandas as pd
import numpy as np
from collections import namedtuple
import json

### constants imports

In [5]:
with open("../constants/columns.json", "r") as file: columns, years = json.load(file).values()

columns = namedtuple('Columns', ' '.join(columns.keys()))(**columns)
years = namedtuple('Years', re.sub(r'[\[\],\']','', str([*map(lambda y: f'y{y}', years)])))(*years)

### tools

In [6]:
YYYY = lambda txt, year : txt.replace('[YYYY]', str(year))

## Create namedtuple

In [7]:
Column = namedtuple("Column", "name formula")

# Formulas

### Formulas applier

In [8]:
def calc_columns(df, new_columns):
    for col in new_columns:
        df[col.name] = df.apply(lambda elm: col.formula(elm), axis=1)

### Formulas implementations (if needed)

In [9]:
"""Is Familiar"""

with open("../constants/isfamiliar_data.json", "r") as file: 
      legal_forms, independence_indicators, accionista_tipo = json.load(file).values()

accionista__percent_directo = []

def isFamiliar(elm):
    
	# initial filters
    # 1. consolidation codes !!!!!
    # 2. Ultimate owner is Spain !!!!
    # 3. foreign parent company is the shareholder 
	# 	*(minimun percentage of the chain control from te focal company to its parent company greather than 50%)
	# 4. operating income > 10000 | > 0


	if elm[YYYY(columns.ebit_YYYY, years[-1])] < 10000: return 'unknown'
# ---

	if legal_forms[elm[columns.forma_juridica_detallada]] == "non_familiar" : return 'no'


	tmp = str(elm[columns.accionista__percent_directo]).split(SEPARATOR)
	index = np.argmax(tmp)
	percent = tmp[index]
	
	try:
		percent = float(percent.replace(',', '.'))
	except:
		if percent == '>50,00': percent = 51.00
		elif percent == '>75,00': percent = 76.00
		else:
			accionista__percent_directo.append(percent)
			return 'unknown'
	
	tipo = str(elm[columns.accionista__tipo]).split(SEPARATOR)[index] 
	#print(elm.name, elm.Nombre, tipo, percent, index, elm[columns.bvd])
	match elm[columns.bvd] :
		case 'C' | 'D': 
			if percent >= 50.01: 
				match accionista_tipo[tipo]:
					case 'familiar': return 'yes'
					case 'non_familiar': return 'no'
		case 'B+' | 'B-': 
			if percent >= 25.01:
				match accionista_tipo[tipo]:
					case 'familiar': return 'yes'
					case 'non_familiar': return 'no'
		case 'A': 
			if percent >= 25.01:
				match accionista_tipo[tipo]:
					case 'familiar': return 'yes'
					case 'non_familiar': return 'no'
		case _: return 'unknown'

	return 'unknown'


In [10]:
def z_zones(z):
    if z > 2.6: return "green"
    elif z >= 1.1: return "grey"
    else: return "red"

def z_zone_cat(z):
    match z:
        case "green": return 0
        case "grey": return 0
        case "red": return 1

def family_cats(fam):
    match fam:
        case "yes": return 1
        case "no": return 0
        case "unknown": return -1

In [11]:
not_nan_filters = [
    *[YYYY(columns.activo_corriente_YYYY, year) for year in years],
    *[YYYY(columns.activo_no_corriente_YYYY, year) for year in years],
    *[YYYY(columns.pasivo_corriente_YYYY, year) for year in years],
    *[YYYY(columns.pasivo_no_corriente_YYYY, year) for year in years],
    
	*[YYYY(columns.importe_neto_de_la_cifra_de_negocios_YYYY, year) for year in years]
    ]

nan_to_zero = [
    *[YYYY(columns.ganancias_acumuladas_YYYY, year) for year in years],
    
	*[YYYY(columns.resultado_ejercicio_YYYY, year) for year in years],
    *[YYYY(columns.dividendos_YYYY, year) for year in years],
    *[YYYY(columns.ebit_YYYY, year) for year in years],
    *[YYYY(columns.ebitda_YYYY, year) for year in years]
]

not_zero = [
     *[YYYY(columns.total_activo_YYYY, year) for year in years]
]

### Formulas array

In [12]:
new_columns = [
	# z components prevs
    *[Column(f"Ganancias retenidas {year}", lambda df, year=year: 
             df[YYYY(columns.ganancias_acumuladas_YYYY, year)] + df[YYYY(columns.resultado_ejercicio_YYYY, year)] + df[YYYY(columns.dividendos_YYYY, year)]) for year in years],
    *[Column(f"Pasivo total {year}", lambda df, year=year: df[YYYY(columns.pasivo_corriente_YYYY, year)] + df[YYYY(columns.pasivo_no_corriente_YYYY, year)]) for year in years],
    
	# indexes components
    #*[Column(f"TA_{year}", lambda df, year=year: df[YYYY(columns.activo_corriente_YYYY, year)]+df[YYYY(columns.activo_no_corriente_YYYY, year)]) for year in years],
    *[Column(f"TD {year}", lambda df, year=year: df[YYYY(columns.pasivo_corriente_YYYY, year)]+df[YYYY(columns.pasivo_no_corriente_YYYY, year)]) for year in years],
    *[Column(f"WC {year}", lambda df, year=year: df[YYYY(columns.activo_corriente_YYYY, year)]-df[YYYY(columns.pasivo_corriente_YYYY, year)]) for year in years],
    
	# z components Xn
	*[Column(f"Z X1 {year}", lambda df, year=year: df[YYYY(columns.WC_YYYY , year)]/df[YYYY(columns.total_activo_YYYY, year)]) for year in years],
    *[Column(f"Z X2 {year}", lambda df, year=year: df[YYYY(columns.ganancias_retenidas_YYYY, year)]/df[YYYY(columns.total_activo_YYYY, year)]) for year in years],
    *[Column(f"Z X3 {year}", lambda df, year=year: df[YYYY(columns.ebit_YYYY, year)]/df[YYYY(columns.total_activo_YYYY, year)]) for year in years],
    *[Column(f"Z X4 {year}", lambda df, year=year: df[YYYY(columns.patrimonio_neto_YYYY, year)]/df[YYYY(columns.total_pasivo_YYYY, year)]) for year in years],
    *[Column(f"Z X5 {year}", lambda df, year=year: df[YYYY(columns.importe_neto_de_la_cifra_de_negocios_YYYY, year)]/df[YYYY(columns.total_activo_YYYY, year)]) for year in years],
      
	# z altman
	*[Column(f"Z {year}", lambda df, year=year: 
          .717*df[YYYY(columns.zX1_YYYY, year)] + .847*df[YYYY(columns.zX2_YYYY, year)] + 3.107*df[YYYY(columns.zX3_YYYY, year)] + .42*df[YYYY(columns.zX4_YYYY, year)] + .998*df[YYYY(columns.zX5_YYYY, year)]
          ) for year in years],
          
	# z altman zones
    #*[Column(f"Z Green Zone {year}", lambda df, year=year: 1 if z_zones(df[YYYY(columns.z_YYYY, year)]) == "green" else 0) for year in years],
    #*[Column(f"Z Grey Zone {year}", lambda df, year=year: 1 if z_zones(df[YYYY(columns.z_YYYY, year)]) == "grey" else 0) for year in years],
    #*[Column(f"Z Red Zone {year}", lambda df, year=year: 1 if z_zones(df[YYYY(columns.z_YYYY, year)]) == "red" else 0) for year in years],
    *[Column(f"Z Cat {year}", lambda df, year=year: z_zone_cat(z_zones(df[YYYY(columns.z_YYYY, year)]))) for year in years], 
    
	# indexes
    *[Column(f"TD/TA {year}", lambda df, year=year: df[YYYY(columns.TD_YYYY, year)]/df[YYYY(columns.total_activo_YYYY, year)]) for year in years],
    *[Column(f"WC/TA {year}", lambda df, year=year: df[YYYY(columns.WC_YYYY, year)]/df[YYYY(columns.total_activo_YYYY, year)]) for year in years],
    *[Column(f"EBITDA/TA {year}", lambda df, year=year: df[YYYY(columns.ebitda_YYYY, year)]/df[YYYY(columns.total_activo_YYYY, year)]) for year in years],
    *[Column(f"ln(TA) {year}", lambda df, year=year: np.log(df[YYYY(columns.total_activo_YYYY, year)])) for year in years],
     Column("isFamiliar", lambda df: family_cats(isFamiliar(df)))
]

# Main

### Import dataframe

In [13]:
df = pd.read_excel(IN_FILENAME)

In [14]:
df[YYYY(columns.ebit_YYYY, 2024)]

0        12006425
1      7763514.86
2        31240000
3         3336769
4      2329233.35
          ...    
865       5107.09
866      2049.889
867        -449.3
868          n.d.
869          n.d.
Name: EBIT\nEUR\n2024, Length: 870, dtype: object

In [15]:
print(nan_to_zero)

['Ganancias acumuladas\nEUR\n2022', 'Ganancias acumuladas\nEUR\n2024', 'Resultado del Ejercicio\nEUR\n2022', 'Resultado del Ejercicio\nEUR\n2024', 'Dividendos (-)\nEUR\n2022', 'Dividendos (-)\nEUR\n2024', 'EBIT\nEUR\n2022', 'EBIT\nEUR\n2024', 'EBITDA\nEUR\n2022', 'EBITDA\nEUR\n2024']


In [16]:
# aqui esta el problema

for f in nan_to_zero: df[f] = df[f].replace('n.d.', 0.0)
df.shape

C:\Users\nelso\AppData\Local\Temp\ipykernel_30648\925931245.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  for f in nan_to_zero: df[f] = df[f].replace('n.d.', 0.0)


(870, 42)

In [17]:
for f in not_nan_filters: df = df[df[f] != 'n.d.']
df.shape

(620, 42)

In [18]:
df[YYYY(columns.total_activo_YYYY, 2022)]

0      3.630959e+08
1      1.177469e+08
2      1.271560e+08
3      9.651429e+07
4      5.961892e+07
           ...     
845    9.074995e+04
848    2.865068e+04
849    1.556881e+06
858    2.170980e+05
861    8.128639e+04
Name: Total activo\nEUR\n2022, Length: 620, dtype: float64

In [19]:
for f in not_nan_filters: df[f] = df[f].astype(float)
for f in not_zero: df[f] = df[f].astype(float)
for f in nan_to_zero: df[f] = df[f].astype(float)
df.shape

(620, 42)

In [20]:
df[YYYY(columns.total_activo_YYYY, 2024)]

0      3.710083e+08
1      1.042030e+08
2      1.264330e+08
3      9.869332e+07
4      6.858963e+07
           ...     
845    7.689615e+04
848    3.198410e+04
849    1.045511e+06
858    2.215469e+05
861    8.126726e+04
Name: Total activo\nEUR\n2024, Length: 620, dtype: float64

In [21]:
for f in not_zero: df = df[df[f] != 0.0]
df.shape

(620, 42)

In [22]:
for elm in [*[df[YYYY(columns.total_activo_YYYY, year)] for year in years]]:
    if (elm == 0).any():
        print("Oh no")
        break

### Calc new columns

In [23]:
calc_columns(df, new_columns)
np.unique(accionista__percent_directo)

array(['-', 'NG'], dtype='<U2')

In [24]:
df['isFamiliar'].value_counts(normalize=True) * 100

isFamiliar
-1    67.580645
 1    23.387097
 0     9.032258
Name: proportion, dtype: float64

In [25]:
df.shape

(620, 73)

### Save dataframe

In [26]:
match fmt:
    case "xlsx": df.to_excel(OUT_FILENAME, index=False)
    case "csv": df.to_csv(OUT_FILENAME, index=False)
    case _: raise Exception("Stop touching things!!")